# Preprocessing

In [ ]:
import pandas as pd
import re
import spacy

In [ ]:
import nltk
import os

# Utilise le répertoire de l'utilisateur (ex: /Users/ton_nom/nltk_data)
nltk_path = os.path.join(os.path.expanduser('~'), 'nltk_data')

# Téléchargement des données
nltk.download('punkt', download_dir=nltk_path)
nltk.download('stopwords', download_dir=nltk_path)
nltk.download('wordnet', download_dir=nltk_path)
nltk.download('omw-1.4', download_dir=nltk_path)

# Optionnel : pour que nltk sache où chercher ces ressources
nltk.data.path.append(nltk_path)

In [ ]:
# Chargement du modèle spaCy
nlp = spacy.load("en_core_web_sm")

In [ ]:
# Chargement du dataset brut
file_path = "./data/dataset.csv"
df = pd.read_csv(file_path)
df['humor'] = df['humor'].astype(bool)

In [ ]:
# Nettoyage de base
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'http\S+|www\S+', '', text)
    text = re.sub(r'[^a-z\s]', '', text)
    return text

df['text_clean'] = df['text'].apply(clean_text)

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [ ]:
nlp = spacy.load("en_core_web_sm")

def preprocess_text_spacy(text):
    doc = nlp(text)
    tokens = [
        token.lemma_.lower()
        for token in doc
        if not token.is_stop and not token.is_punct and token.is_alpha
    ]
    return " ".join(tokens)

In [ ]:
from tqdm.notebook import tqdm
tqdm.pandas()

df['text_processed'] = df['text_clean'].progress_apply(preprocess_text_spacy)

In [ ]:
# Calcul de la longueur des textes
df['text_length'] = df['text'].apply(lambda x: len(str(x).split()))

assert 'text' in df.columns, "Colonne 'text' manquante."
assert 'humor' in df.columns, " Colonne 'humor' manquante."
assert pd.api.types.is_bool_dtype(df['humor']), "Colonne 'humor' n'est pas booléenne."
assert 'text_clean' in df.columns, "Colonne 'text_clean' manquante."
assert df['text_clean'].str.len().gt(0).all(), "Certaines lignes de 'text_clean' sont vides."
assert 'text_length' in df.columns, "Colonne 'text_length' manquante."
assert df['text_length'].gt(0).all(), "Certaines longueurs de texte sont nulles."

print(" Toutes les étapes principales du EDA sont validées avec succès ! ")

In [ ]:
# Sauvegarde du dataset nettoyé

output_path = "data/clean_dataset.csv"
df.to_csv(output_path, index=False)

print(f" Dataset nettoyé sauvegardé dans : {output_path}")